Amazon Recommendation System uygulamamız için modelimizi kaydedeceğiz

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

In [3]:
# VERİYİ YÜKLE

In [4]:
df = pd.read_csv("ratings_Electronics (1).csv", header=None)

In [5]:
df.head()

,0,1,2,3
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7824482 entries, 0 to 7824481
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   0       object 
 1   1       object 
 2   2       float64
 3   3       int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 238.8+ MB


In [7]:
df.isnull().sum()

0    0
1    0
2    0
3    0
dtype: int64

In [8]:
df.columns = ['userId', 'productId', 'rating', 'timestamp']

In [9]:
df = df[['userId', 'productId', 'rating']]

In [10]:
df.head()

,userId,productId,rating
0,AKM1MP6P0OYPR,0132793040,5.0
1,A2CX7LUOHB2NDG,0321732944,5.0
2,A2NWSAGRHCP8N5,0439886341,1.0
3,A2WNBOD3WNDNKT,0439886341,3.0
4,A1GI0U4ZRJA8WN,0439886341,1.0


In [11]:
print("Orijinal veri boyutu:", df.shape)

Orijinal veri boyutu: (7824482, 3)


In [12]:
# FİLTRELEME

In [13]:
# Aktif kullanıcılar (en az 10 rating)

In [14]:
active_users = df['userId'].value_counts()

In [15]:
active_users = active_users[active_users >= 10].index

In [16]:
df = df[df['userId'].isin(active_users)]

In [17]:
# Popüler ürünler (en az 10 rating)

In [18]:
popular_products = df['productId'].value_counts()

In [19]:
popular_products = popular_products[popular_products >= 10].index

In [20]:
df = df[df['productId'].isin(popular_products)]

In [21]:
print("Filtrelenmiş veri boyutu:", df.shape)

Filtrelenmiş veri boyutu: (722237, 3)


In [22]:
# TRAIN / TEST AYRIMI

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [25]:
# ITEM–USER SPARSE MATRIX

In [26]:
user_codes = train_df['userId'].astype("category").cat.codes

In [27]:
# Kategorik kodlama

In [28]:
product_codes = train_df['productId'].astype("category").cat.codes

In [29]:
user_categories = train_df['userId'].astype("category").cat.categories

In [30]:
product_categories = train_df['productId'].astype("category").cat.categories

In [31]:
from scipy.sparse import csr_matrix

In [32]:
item_user_matrix = csr_matrix((train_df['rating'], (product_codes, user_codes)))

In [33]:
print("Sparse matrix shape:", item_user_matrix.shape)

Sparse matrix shape: (20972, 62991)


In [34]:
# ITEM–ITEM COSINE SIMILARITY

In [35]:
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
item_similarity = cosine_similarity(item_user_matrix, dense_output=False)

In [37]:
# ITEM-BASED TAHMİN FONKSİYONU

In [38]:
def predict_rating_item_based(user_id, product_id):
    if user_id not in user_categories or product_id not in product_categories:
        return np.nan

    user_idx = user_categories.get_loc(user_id)
    product_idx = product_categories.get_loc(product_id)

    similarities = item_similarity[product_idx].toarray().flatten()

    user_ratings = item_user_matrix[:, user_idx].toarray().flatten()

    mask = user_ratings > 0
    if mask.sum() == 0:
        return np.nan

    return np.dot(similarities[mask], user_ratings[mask]) / np.sum(similarities[mask])

In [39]:
# MODEL SKORU (RMSE)

In [40]:
y_true = []

In [41]:
y_pred = []

In [42]:
import numpy as np

In [43]:
from tqdm import tqdm

In [44]:
for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    pred = predict_rating_item_based(
        row['userId'],
        row['productId']
    )
    if not np.isnan(pred):
        y_true.append(row['rating'])
        y_pred.append(pred)

100%|█████████████████████████████████████████████████████████████████████████| 144448/144448 [02:56<00:00, 818.62it/s]


In [45]:
from sklearn.metrics import mean_squared_error

In [46]:
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

In [47]:
print("\n📊 MODEL RMSE:", rmse)


📊 MODEL RMSE: 1.2271461280859346


In [48]:
# MODEL SAVE

In [49]:
import pickle

In [50]:
with open("amazon_item_based_model.pkl", "wb") as f:
    pickle.dump(
        {
            "item_user_matrix": item_user_matrix,
            "item_similarity": item_similarity,
            "user_categories": user_categories,
            "product_categories": product_categories
        },
        f
    )